## Метрики качества для сравнения итоговых графов

Итоговая реализация в **final_metrics.py**

Используем GED и поверх него добавляем штраф за несовпадение меток в вершинах / узлах

На вход подаются 2 сцены вида

```
{
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}
```

### full_ged_score(scene_1, scene_2)

результат - от 0 до 1. 1- полное совпадение 0 - отсутствие сходства

### Как считается

считаем, сколько нужно сделать изменений (минимальнео число) включая переименование вершин и ребер (если они именованы) чтобы получить из первого графа второй. 

In [68]:
import networkx as nx
from networkx.algorithms.similarity import optimize_graph_edit_distance

from typing import Dict

import spacy
nlp = spacy.load("ru_core_news_sm")

## Лемматизация сцены 

(будет полезно сделать на всякий случай)

лемматизирует объекты, атрбуты и объекты в триплетах

In [69]:
def lemmatize_scene(scene: Dict, nlp) -> Dict:
    def lemmatize(text: str) -> str:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

    new_scene = {
        "scene": {
            "location": scene["scene"].get("location", ""),
            "objects": [],
            "relations": []
        }
    }

    # Лемматизируем объекты и их признаки
    for obj in scene["scene"].get("objects", []):
        for obj_name, attributes in obj.items():
            obj_lemma = lemmatize(obj_name)
            attrs_lemma = [lemmatize(attr) for attr in attributes]
            new_scene["scene"]["objects"].append({obj_lemma: attrs_lemma})

    # Лемматизируем связи
    for subj, rel, obj in scene["scene"].get("relations", []):
        subj_lemma = lemmatize(subj)
        rel_lemma = lemmatize(rel)
        obj_lemma = lemmatize(obj)
        new_scene["scene"]["relations"].append([subj_lemma, rel_lemma, obj_lemma])

    return new_scene

In [70]:
scene = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"Гаечного ключа": ["металлического"]},
            {"домкратом": ["металлическим", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["Аккумулятора", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкратом"]
        ]
    }
}


In [71]:
lemmatize_scene(scene, nlp)

{'scene': {'location': 'автосервис',
  'objects': [{'гаечный ключ': ['металлический']},
   {'домкрат': ['металлический', 'тяжёлый', 'прочный']},
   {'аккумулятор': []}],
  'relations': [['аккумулятор', 'рядом с', 'гаечный ключ'],
   ['гаечный ключ', 'на', 'домкрат']]}}

### Реализация GED

In [72]:
def scene_to_graph_with_attrs(scene: Dict) -> nx.DiGraph:
    G = nx.DiGraph()

    # Вершины: объекты и атрибуты
    for obj in scene["scene"]["objects"]:
        for obj_name, attributes in obj.items():
            G.add_node(obj_name, type="object")
            for attr in attributes:
                G.add_node(attr, type="attribute")
                G.add_edge(obj_name, attr, relation=None)

    # Пространственные отношения
    for subj, rel, obj in scene["scene"]["relations"]:
        G.add_edge(subj, obj, relation=rel)

    return G

In [77]:
# Функция сопоставления меток узлов
# узлы совпадают если в вершине одинаковые объекты
def node_match(n1, n2):
    return n1.get("type") == n2.get("type")


# Функция сопоставления меток рёбер
# если у обоих relation=None — считается совпадением (нормально)
# то есть ребра объект-атрибут в обоих гафах считаются совпадающими даже при условии что нет метки
def edge_match(e1, e2):
    return e1.get("relation") == e2.get("relation")


def evaluate_ged_score(scene1: Dict, scene2: Dict, normalize = True) -> float:
    if normalize:
        nlp = spacy.load("ru_core_news_sm")
        scene1 = lemmatize_scene(scene1, nlp)
        scene2 = lemmatize_scene(scene2, nlp)    
    
    G1 = scene_to_graph_with_attrs(scene1)
    G2 = scene_to_graph_with_attrs(scene2)

    
    # Оптимизируем GED с учётом атрибутов
    # optimize_graph_edit_distance использует итеративный эвристический поиск 
    # по возможным сопоставлениям графов, первая выдача как правило оптимальная
    # (но NP сложная задача и на больших графах работает медленно)
    ged_iter = optimize_graph_edit_distance(G1, G2, node_match=node_match, edge_match=edge_match)

    try:
        edit_distance = next(ged_iter)
    except StopIteration:
        return 0.0

    # Нормализация: максимум — если ни одной вершины и ребра не совпало
    max_size = max(G1.number_of_nodes() + G1.number_of_edges(),
                   G2.number_of_nodes() + G2.number_of_edges())

    if max_size == 0:
        return 1.0  # два пустых графа считаем полностью совпадающими

    similarity = 1.0 - (edit_distance / max_size)
    return {
        "GED_score" : round(similarity, 4)
    }

In [78]:
# Тестовые сцены

scene_ref = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

# Идеальное совпадение
scene_same = scene_ref

# Убрали один атрибут, заменили отношение
scene_minor_diff = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["тяжелый", "прочный"]},  # "металлический" пропущен
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "перед", "гаечный ключ"],  # заменили предлог
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

# Сильное отличие: другие объекты и связи
scene_major_diff = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"вентилятор": ["старый"]},
            {"домкрат": ["ржавый"]},
            {"инструмент": ["маленький"]}
        ],
        "relations": [
            ["вентилятор", "под", "домкрат"]
        ]
    }
}

# Ошибка в типе узла: один из атрибутов стал объектом
scene_type_mismatch = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"станок": []},  # признак стал объектом
            {"домкрат": ["тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "металлический"],
            ["металлический", "на", "домкрат"]
        ]
    }
}

# Собираем результаты
# Посчитаем для всех 4 сцен

print("perfect_match:", evaluate_ged_score(scene_ref, scene_same))
print("minor_difference:", evaluate_ged_score(scene_ref, scene_minor_diff))
print("major_difference:", evaluate_ged_score(scene_ref, scene_major_diff))
print("type_mismatch:", evaluate_ged_score(scene_ref, scene_type_mismatch))




perfect_match: {'GED_score': 1.0}
minor_difference: {'GED_score': 0.8333}
major_difference: {'GED_score': 0.5833}
type_mismatch: {'GED_score': 0.4167}


In [79]:
scene_1 = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

scene_2 = {
    "scene": {
        "location": "автосервис",
        "objects": [
            {"гаечный ключ": ["металлический"]},
            {"домкрат": ["металлический", "тяжелый", "прочный"]},
            {"аккумулятор": []}
        ],
        "relations": [
            ["аккумулятор", "рядом с", "гаечный ключ"],
            ["гаечный ключ", "на", "домкрат"]
        ]
    }
}

evaluate_ged_score(scene_1, scene_2)

{'GED_score': 1.0}